In [ ]:
import pandas as pd
import re
import glob
import os

folder = '/content/'
extraction_number = '06'

KEYWORDS = [
    'divulgação científica', 'divulgacao cientifica', 'popularização da ciência',
    'popularizacao da ciencia', 'ensino de ciência', 'educação científica',
    'educacao cientifica', 'ciência para crianças', 'ciencia para criancas',
    'curiosidades científicas', 'curiosidades cientificas', 'ciencion',
    'revista ciência', 'revista galileu', 'superinteressante', 'ciência hoje',
    'ciencia hoje', 'blog', 'contém química', 'dragões de garagem',
    'guia dos entusiastas', 'ciência viva', 'scicast', 'ciência aberta',
    'jornal da usp', 'comciência', 'museus de ciência', 'museu de ciência',
    'jornalismo científico', 'alfabetização científica', 'texto de divulgação',
    'postagem', 'podcast', 'rádio', 'entrevista', 'programa', 'mesa redonda',
    'live', 'vídeo', 'youtube', 'canal', 'tv', 'documentário', 'série',
    'reportagem', 'webinar', 'rede social', 'spotify', 'instagram',
    'facebook'
]

def find_keyword(input_text):
    if not isinstance(input_text, str):
        return False
    input_text = input_text.lower()
    return any(p in input_text for p in KEYWORDS)

csv_files = glob.glob(os.path.join(folder, '*.csv'))
dfs_dc = []

for path in csv_files:
    if path.startswith(f'{folder}{extraction_number} - '):
        print(f'path: {path}')
        df = pd.read_csv(path, sep=None, engine='python', encoding='utf-8')
        df['text_join'] = df.apply(lambda x: ' '.join(str(v) for v in x), axis=1)
        df_dc = df[df['text_join'].apply(find_keyword)].drop(columns=['text_join'])

        if not df_dc.empty:
            df_dc['source_file'] = os.path.basename(path)
            dfs_dc.append(df_dc)

if dfs_dc:
    df_dc_output = pd.concat(dfs_dc, ignore_index=True)
    output_path = os.path.join(folder, f'output - {extraction_number}.csv')
    df_dc_output.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
    print(f'output_path: {output_path}')
    print(f'total rows: {len(df_dc_output)}')

    if 'Autores' in df_dc_output.columns:
        autores_explode = (
            df_dc_output['Autores']
            .dropna()
            .astype(str)
            .str.replace(r'\s*;\s*', ';', regex=True)
            .str.split(';')
            .explode()
            .str.strip()
        )

        autores_explode = autores_explode[autores_explode != ""]
        autores_explode = (
            autores_explode
            .str.lower()
            .str.replace(r'\(org\.\)', '', regex=True)
            .str.replace(r'\borgs?\b', '', regex=True)
            .str.replace(r'[\.,]', '', regex=True)
            .str.replace(r'\s+', ' ', regex=True)
            .str.strip()
        )

        autores_explode = autores_explode[autores_explode != ""]
        autor_counts = autores_explode.value_counts().reset_index()
        autor_counts.columns = ['Autor', 'Count']

        autor_counts = autor_counts.sort_values(by='Count', ascending=False).reset_index(drop=True)

        authors_path = os.path.join(folder, f'distinct_authors - {extraction_number}.csv')
        autor_counts.to_csv(authors_path, index=False, sep=';', encoding='utf-8-sig')

        print(f'authors_path: {authors_path}')
        print(f'total unique authors: {len(autor_counts)}')